# Auto Builder
AutoGen offers conversable agents powered by LLM, tool, or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation.
Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

In this notebook, we introduce a new class, `AgentCreator`, to help user build an automatic task solving process powered by multi-agent system. Specifically, our building pipeline include `init()`, `build()`, and `start()`. In `build()`, we prompt a gpt-4 model to create multiple participant agent and initialize a group chat, and specify whether this task need programming to solve. After that, user can call `start()` at a proper time to complete the task. 

## Requirement

Auto Agent setup need `vllm>=0.2.0` and `fastchat>=0.2.30`, you can use the following command to install the latest version of [vLLM](https://github.com/vllm-project/vllm) and [FastChat](https://github.com/lm-sys/FastChat).
```
pip install vllm "fastchat[model_worker]"
```

In [ ]:
!pip install vllm "fastchat[model_worker]"

## Specify a task with detailed information
Describe your task in natural language and specify your target (e.g., find some paper from arxiv).

In [1]:
task="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."

(Optional) you can specify a building task in natural language to let build manager (a LLM) build a specific group of agent for your task (e.g., "Create a product_manager and coder for me, use gpt-3.5-turbo as backbone."), or just use the same task as building task and let build manager to decide which agent should be built for your task.

In [2]:
building_task="Find a latest paper about gpt-4 on arxiv and find its potential applications in software."

## Prepare configuration
Prepare a `config_path` for assistant agent to limit the choice of LLM you want to use in this task. This config can be a path of json file or a name of environment variable. A `default_llm_config` is also required for initialize the specific config of LLMs like seed, temperature, etc...

In [3]:
config_path = '/home/elpis_ubuntu/LLM/autogen/OAI_CONFIG_LIST'
default_llm_config = {
    'temperature': 0
}

## Complete the task building
Then, we can use the `AgentCreator` to automatically complete the above task. First, initialize the AgentCreator with the predefined `task` and `config_path`. Then, call the `build` with the `default_llm_config`. GPT-4 will automatically create multiple agents and initialize a groupchat that use to complete this task. Finally, call the `start()` to start the task completion process with a groupchat. When the task begin, groupchat content between each agent will be shown in the console. This process will become interactive when the groupchat manager think that is necessary. Note that if this process involve some open-sourced LLMs (for example, LLaMA 2), an endpoint server will be automatically set up with a background process. You can call the `clear_all()` to wipe them out after the task completed.

In [4]:
from autobuild import AgentCreator

administrator = AgentCreator(config_path=config_path)
administrator.build(task, default_llm_config)
administrator.start(task)
administrator.clear_all_agents()

Initializing usable port...
57510 ports found.
User_proxy (to chat_manager):

Find a latest paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------
Coder_gpt_35 (to chat_manager):

To find the latest paper about GPT-4 on arXiv, we can use the arXiv API to search for papers related to GPT-4. Then, we can analyze the paper to identify its potential applications in software.

Here is the Python code to accomplish this task:

```python
import requests

# Search for papers related to GPT-4 on arXiv
search_query = "GPT-4"
api_url = f"http://export.arxiv.org/api/query?search_query=all:{search_query}&sort=submittedDate&order=descending&max_results=1"
response = requests.get(api_url)

# Parse the XML response and extract the paper details
xml_data = response.text
paper_title = xml_data.split("<title>")[2].split("</title>")[0]
paper_summary = xml_data.split("<summary>")[1].split("</summary>")[0]

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


User_proxy (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 10, in <module>
    paper_title = xml_data.split("<title>")[2].split("</title>")[0]
IndexError: list index out of range


--------------------------------------------------------------------------------
Coder_gpt_35 (to chat_manager):

I apologize for the error. It seems that the code failed to extract the paper details from the XML response. This could be due to changes in the XML structure or the absence of the expected tags.

To resolve this issue, we can use the `feedparser` library to parse the XML response and extract the paper details. Here's the updated code:

```python
import requests
import feedparser

# Search for papers related to GPT-4 on arXiv
search_query = "GPT-4"
api_url = f"http://export.arxiv.org/api/query?search_query=all:{search_query}&sort=submittedDate&order=descending&max_results=1"
response = requests.get(api_url)

# Parse the XML respo

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


User_proxy (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
Latest Paper about GPT-4 on arXiv:
Title: Can LLMs like GPT-4 outperform traditional AI tools in dementia
  diagnosis? Maybe, but not today
Summary: Recent investigations show that large language models (LLMs), specifically
GPT-4, not only have remarkable capabilities in common Natural Language
Processing (NLP) tasks but also exhibit human-level performance on various
professional and academic benchmarks. However, whether GPT-4 can be directly
used in practical applications and replace traditional artificial intelligence
(AI) tools in specialized domains requires further experimental validation. In
this paper, we explore the potential of LLMs such as GPT-4 to outperform
traditional AI tools in dementia diagnosis. Comprehensive comparisons between
GPT-4 and traditional AI tools are conducted to examine their diagnostic
accuracy in a clinical setting. Experimental results on two real clinical
datasets show tha

## (Optional) Save configs

You can save the building config by calling `save()` function. This function will save all necessary information for building all group chat agents. You can provide a specific filename for this function. If not, it will save config to the local path with a generated filename 'save_config_TASK_MD5.json'.

In [5]:
saved_path = administrator.save()

Building config saved to ./save_config_5e043e06bd30f3593fdaf15e8b594002.json


Once you save the building config, you can call `load()` function with the path of the saved config to skip the building process (note that building process will call online LLMs' api).

In [6]:
new_administrator = AgentCreator(config_path=config_path)
new_administrator.load(saved_path)
new_administrator.start(task)
new_administrator.clear_all_agents()

Initializing usable port...
115020 ports found.
User_proxy (to chat_manager):
Find a latest paper about gpt-4 on arxiv and find its potential applications in software.

--------------------------------------------------------------------------------
Coder_gpt_35 (to chat_manager):

To find the latest paper about GPT-4 on arXiv, we can use the arXiv API to search for papers related to GPT-4. Then, we can analyze the paper to identify its potential applications in software.

Here is the Python code to accomplish this task:

```python
import requests

# Search for papers related to GPT-4 on arXiv
search_query = "GPT-4"
api_url = f"http://export.arxiv.org/api/query?search_query=all:{search_query}&sort=submittedDate&order=descending&max_results=1"
response = requests.get(api_url)

# Parse the XML response and extract the paper details
xml_data = response.text
paper_title = xml_data.split("<title>")[2].split("</title>")[0]
paper_summary = xml_data.split("<summary>")[1].split("</summary>")[0]

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


User_proxy (to chat_manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 10, in <module>
    paper_title = xml_data.split("<title>")[2].split("</title>")[0]
IndexError: list index out of range


--------------------------------------------------------------------------------
Coder_gpt_35 (to chat_manager):

I apologize for the error. It seems that the code failed to extract the paper details from the XML response. This could be due to changes in the XML structure or the absence of the expected tags.

To resolve this issue, we can use the `feedparser` library to parse the XML response and extract the paper details. Here's the updated code:

```python
import requests
import feedparser

# Search for papers related to GPT-4 on arXiv
search_query = "GPT-4"
api_url = f"http://export.arxiv.org/api/query?search_query=all:{search_query}&sort=submittedDate&order=descending&max_results=1"
response = requests.get(api_url)

# Parse the XML respo

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


User_proxy (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 
Latest Paper about GPT-4 on arXiv:
Title: Can LLMs like GPT-4 outperform traditional AI tools in dementia
  diagnosis? Maybe, but not today
Summary: Recent investigations show that large language models (LLMs), specifically
GPT-4, not only have remarkable capabilities in common Natural Language
Processing (NLP) tasks but also exhibit human-level performance on various
professional and academic benchmarks. However, whether GPT-4 can be directly
used in practical applications and replace traditional artificial intelligence
(AI) tools in specialized domains requires further experimental validation. In
this paper, we explore the potential of LLMs such as GPT-4 to outperform
traditional AI tools in dementia diagnosis. Comprehensive comparisons between
GPT-4 and traditional AI tools are conducted to examine their diagnostic
accuracy in a clinical setting. Experimental results on two real clinical
datasets show tha